In [1]:
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, _tree, export_graphviz, export_text
np.set_printoptions(suppress=True)

warnings.filterwarnings(action='ignore')

In [2]:
# sage, srgnn 중 선택
embed_model = 'srgnn'

filename = {'sage' : 'user_embedding_sage_sid_6to6_weighted.npy', 'srgnn' : 'srgnn_user_hybrid_6to6_weighted.npy'}

In [3]:
data_path = "../data"
embedding = np.load(os.path.join(data_path, filename[embed_model])) # 불러올 임베딩 파일 이름
view_hist = pd.read_pickle(os.path.join(data_path,'sid_6to6.pkl')) # 불러올 시청 이력 파일
name_file = pickle.load(open(os.path.join(data_path,'3com_prg_dict.pkl'), 'rb'))
k = 15                #여기서 클러스터 개수 정하기

In [4]:
x = np.where(np.sum(embedding, axis=1)==0)
x = x[0]
view_hist.MAC = view_hist.MAC.astype(int)
view_hist = view_hist[~view_hist['MAC'].isin(x)]

In [5]:
df = view_hist.copy()
df

,MAC,start_time,end_time,stay_time,interval_time,ptitle,ptitle_prep,genre1,genre2,grade,ch_name,sid_6to6
0,0,1661748353,1661748451,98,0.0,channelollehtv,channelollehtv,NaN,NaN,NaN,channelollehtv,0_0
1,0,1661748458,1661748542,84,7.0,sbs,sbs,NaN,NaN,NaN,sbs,0_0
2,0,1661748542,1661785199,36657,0.0,롯데홈쇼핑,롯데홈쇼핑,NaN,NaN,NaN,롯데홈쇼핑,0_0
3,0,1661853365,1661869575,16210,68166.0,롯데홈쇼핑,롯데홈쇼핑,NaN,NaN,NaN,롯데홈쇼핑,0_1
4,0,1661911317,1661971739,60422,41742.0,channelollehtv,channelollehtv,NaN,NaN,NaN,channelollehtv,0_2
...,...,...,...,...,...,...,...,...,...,...,...,...
31782286,4846,1667220300,1667220469,169,0.0,이데베논 앰플 시즌3,홈쇼핑_뷰티,L0,L007,0.0,롯데홈쇼핑,4846_152
31782290,4846,1667220484,1667220694,210,1.0,토트넘 이지웨어,홈쇼핑_건강,L0,L004,0.0,skstoa02,4846_152
31782293,4846,1667220700,1667221093,393,0.0,[특집]TV CHOSUN 뉴스9,뉴스_일반,I0,I007,0.0,tv조선,4846_152
31782294,4846,1667221093,1667221200,107,0.0,특집 KBS 뉴스 9,뉴스_뉴스,I0,I001,0.0,kbs1,4846_152


In [6]:
# stay_time 초 단위에서 분 단위로 바꾸기
df['stay_time'] = df['stay_time'].apply(lambda x: round(x/60, 1))
df.head()

,MAC,start_time,end_time,stay_time,interval_time,ptitle,ptitle_prep,genre1,genre2,grade,ch_name,sid_6to6
0,0,1661748353,1661748451,1.6,0.0,channelollehtv,channelollehtv,NaN,NaN,NaN,channelollehtv,0_0
1,0,1661748458,1661748542,1.4,7.0,sbs,sbs,NaN,NaN,NaN,sbs,0_0
2,0,1661748542,1661785199,611.0,0.0,롯데홈쇼핑,롯데홈쇼핑,NaN,NaN,NaN,롯데홈쇼핑,0_0
3,0,1661853365,1661869575,270.2,68166.0,롯데홈쇼핑,롯데홈쇼핑,NaN,NaN,NaN,롯데홈쇼핑,0_1
4,0,1661911317,1661971739,1007.0,41742.0,channelollehtv,channelollehtv,NaN,NaN,NaN,channelollehtv,0_2


In [7]:
# prog_aud
# 평균 표준편차 구하기
prog_mean = df.groupby('ptitle_prep')['stay_time'].mean()
prog_std = df.groupby('ptitle_prep')['stay_time'].std()

# merge를 위한 column 이름 바꾸기
prog_mean.name = 'mean'
prog_std.name = 'std'

# mean merge
merged_df = pd.merge(df, prog_mean, on='ptitle_prep', how='left')

# std merge
merged_df = pd.merge(merged_df, prog_std, on='ptitle_prep', how='left')
merged_df

,MAC,start_time,end_time,stay_time,interval_time,ptitle,ptitle_prep,genre1,genre2,grade,ch_name,sid_6to6,mean,std
0,0,1661748353,1661748451,1.6,0.0,channelollehtv,channelollehtv,NaN,NaN,NaN,channelollehtv,0_0,45.395354,125.438502
1,0,1661748458,1661748542,1.4,7.0,sbs,sbs,NaN,NaN,NaN,sbs,0_0,36.481975,94.745507
2,0,1661748542,1661785199,611.0,0.0,롯데홈쇼핑,롯데홈쇼핑,NaN,NaN,NaN,롯데홈쇼핑,0_0,11.101388,65.199511
3,0,1661853365,1661869575,270.2,68166.0,롯데홈쇼핑,롯데홈쇼핑,NaN,NaN,NaN,롯데홈쇼핑,0_1,11.101388,65.199511
4,0,1661911317,1661971739,1007.0,41742.0,channelollehtv,channelollehtv,NaN,NaN,NaN,channelollehtv,0_2,45.395354,125.438502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11305556,4846,1667220300,1667220469,2.8,0.0,이데베논 앰플 시즌3,홈쇼핑_뷰티,L0,L007,0.0,롯데홈쇼핑,4846_152,10.169349,19.354949
11305557,4846,1667220484,1667220694,3.5,1.0,토트넘 이지웨어,홈쇼핑_건강,L0,L004,0.0,skstoa02,4846_152,10.118495,18.926326
11305558,4846,1667220700,1667221093,6.5,0.0,[특집]TV CHOSUN 뉴스9,뉴스_일반,I0,I007,0.0,tv조선,4846_152,19.936198,32.483917
11305559,4846,1667221093,1667221200,1.8,0.0,특집 KBS 뉴스 9,뉴스_뉴스,I0,I001,0.0,kbs1,4846_152,20.720312,28.670295


In [8]:
# 이상치 대체하기 전 각 프로그램별 최대값 확인
outli_max = df[df['ptitle_prep'] == '(i)cj온스타일플러스']
outli_max['stay_time'].max()

896.7

In [9]:
# stay_time이 이상치인 경우 3_sigma로 대체
merged_df['3_sigma'] = merged_df['mean'] + 3*merged_df['std']
merged_df['stay_time'] = merged_df.apply(lambda x: x['3_sigma'] if x['stay_time'] > x['3_sigma'] else x['stay_time'], axis=1)

# 이상치 대체한 최종 데이터프레임
final_df = merged_df[['MAC', 'ptitle_prep', 'stay_time']]
final_df

,MAC,ptitle_prep,stay_time
0,0,channelollehtv,1.600000
1,0,sbs,1.400000
2,0,롯데홈쇼핑,206.699923
3,0,롯데홈쇼핑,206.699923
4,0,channelollehtv,421.710859
...,...,...,...
11305556,4846,홈쇼핑_뷰티,2.800000
11305557,4846,홈쇼핑_건강,3.500000
11305558,4846,뉴스_일반,6.500000
11305559,4846,뉴스_뉴스,1.800000


In [10]:
# 이상치 대체 후 각 프로그램별 최대값 확인
outli_max = final_df[final_df['ptitle_prep'] == '(i)cj온스타일플러스']
outli_max['stay_time'].max()

199.51128233054484

In [11]:
pivot_df = pd.pivot_table(final_df, index='MAC', columns='ptitle_prep', values='stay_time', fill_value=0)[[str(name) for name in name_file.keys()]]
pivot_df

ptitle_prep,(i)cj온스타일플러스,(i)신세계쇼핑,007골드핑거,007골든아이,007나를사랑한스파이,007네버다이,007네버세이네버어게인,007노타임투다이,007다이아몬드는영원히,007두번산다,...,힌덴부르크,힐러걸,힐링다큐한국의섬,힐링로드만남,힐링로드토크멘터리구름위를걷다,힐링스테이지그대에게,힐링스테이지그대에게시즌2bside,힐링이필요해,힘내라우리시장,힘을내요미스터리
MAC,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.1,0.00,0.0,0.0,0.0,0.8,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.15,0.0,0.0,0.0,0.0,0.0
4,7.185714,10.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4842,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.20,0.0,0.0,0.0,0.0,0.0
4843,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4844,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [12]:
pivot_df.to_csv('../data/pivot_ptitle_data.csv')

In [13]:
pd.read_csv('../data/pivot_ptitle_data.csv', index_col=0)

,(i)cj온스타일플러스,(i)신세계쇼핑,007골드핑거,007골든아이,007나를사랑한스파이,007네버다이,007네버세이네버어게인,007노타임투다이,007다이아몬드는영원히,007두번산다,...,힌덴부르크,힐러걸,힐링다큐한국의섬,힐링로드만남,힐링로드토크멘터리구름위를걷다,힐링스테이지그대에게,힐링스테이지그대에게시즌2bside,힐링이필요해,힘내라우리시장,힘을내요미스터리
MAC,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.1,0.00,0.0,0.0,0.0,0.8,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.15,0.0,0.0,0.0,0.0,0.0
4,7.185714,10.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4842,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.20,0.0,0.0,0.0,0.0,0.0
4843,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4844,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
